# Projeto xo-mosquito

> Analise exploratoria de infraestrura dos Bairros de Recife

## Proposta
Levando em consideração o contexto das doenças tropicais negligenciadas, em
especial Dengue, Zika e Chikungunya, e seu impacto social e, segundo IANPHI (2016),
econômico na américa latina, especialmente no Brasil, o projeto possui como proposta:
- Analisar a relação entre o perfil social dos indivíduos afetados e as incidências de doenças transmitidas pelo mosquito Aedes Aegypti;
- Investigar a influência da localidade física e climáticos em que se encontram tais indivíduos;
- Encontrar padrões nos casos relatados para assim sugerir abordagens de vigilância que possam ser implementados por governos.

O objetivo é analisar essas hipóteses e tentar validá-las ou não em relação a Recife,
Pernambuco, mas se não for possível, devido a falta de dados, então o escopo será analisar
globalmente.

## 1. Coleta e limpeza dos dados

In [1]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

Dataset fornecido pela Secretaria de Infraestrutura e Serviços Urbanos de Recife pelo site http://dados.recife.pe.gov.br.

[__Área urbana__](http://dados.recife.pe.gov.br/dataset/area-urbana)

Este conjunto descreve dados sobre a área espacial da cidade do Recife, incluindo dados sobre divisões do espaço físico da cidade, como: bairros, distritos, setores, quadras, faces de quadra, RPAs (Regiões Político Administrativas), micro-regiões, praças, parques, áreas verdes, recursos hídricos, lotes e edificações.

### 1.1 Bairros
Identifica os bairros do Recife, com a localização geográfica e a associação com a RPA (Região Político Administrativa) e a Microregião onde o bairro está inserido.

In [2]:
bairros_resumido = pd.read_csv('http://dados.recife.pe.gov.br/dataset/2a9b1c39-0700-4ddf-9a10-b3c8d5d9396c/resource/ddcf8195-23c0-4ecc-94b5-ec6533a5ca74/download/tabela-de-bairros.csv', encoding = 'latin1', sep=';')

print(bairros_resumido.head(5))

geojson = pd.read_json('http://dados.recife.pe.gov.br/dataset/c1f100f0-f56f-4dd4-9dcc-1aa4da28798a/resource/e43bee60-9448-4d3d-92ff-2378bc3b5b00/download/bairros.geojson')

   Nº Localidade       Nome Localidade Nome Município
0            132               AFLITOS         RECIFE
1            779              AFOGADOS         RECIFE
2            337             AGUA FRIA         RECIFE
3            493         ALTO DO MANDU         RECIFE
4            540   ALTO JOSE BONIFACIO         RECIFE


In [3]:
bairros = json_normalize(geojson.features)
bairros.drop(['type','geometry.type','geometry.coordinates'], axis=1, inplace=True)
bairros = bairros.set_index('id')
bairros.rename(index=str, columns={"properties.bairro_codigo": "codigo", "properties.bairro_nome": "nome", "properties.bairro_nome_ca": "nome_caixa_alta", "properties.microrregiao": "microrregiao", "properties.rpa": "rpa"}, inplace=True)
bairros

,codigo,nome,nome_caixa_alta,microrregiao,rpa
id,,,,,
0,728,Cidade Universitária,CIDADE UNIVERSITARIA,3,4
1,116,Soledade,SOLEDADE,2,1
2,710,Engenho do Meio,ENGENHO DO MEIO,2,4
3,841,Caçote,CACOTE,2,5
4,884,Cohab,COHAB,3,6
5,736,Várzea,VARZEA,3,4
6,701,Torrões,TORROES,2,4
7,698,Iputinga,IPUTINGA,1,4
8,752,Curado,CURADO,1,5


In [4]:
bairros['microrregiao'] = bairros.microrregiao.astype('category')
bairros['rpa'] = bairros.rpa.astype('category')
bairros['nome'] = bairros.nome.astype('str')
bairros['nome_caixa_alta'] = bairros.nome_caixa_alta.astype('str')
bairros.dtypes

codigo                int64
nome                 object
nome_caixa_alta      object
microrregiao       category
rpa                category
dtype: object

### 1.2 Recursos Hídricos

Identifica geograficamente a bacia hidrográfica da cidade do Recife, contendo rios, braços de rios, riachos, bacias portuárias, canais, córregos, açudes, lagoas, alagados na cidade do Recife.

In [5]:
geojson = pd.read_json('data/recursoshidricos/RecursosHidricos.geojson')

In [6]:
hidricos = json_normalize(geojson.features)
hidricos.drop(['type','geometry.type','geometry.coordinates'],axis=1, inplace=True)
hidricos = hidricos.set_index('id')
hidricos.rename(index=str, columns={"properties.nome_recursohidrico": "nome"}, inplace=True)
hidricos.head(5)

,nome
id,
0,RIO CAPIBARIBE
1,RIO CAPIBARIBE
2,RIO CAPIBARIBE
3,RIO CAPIBARIBE
4,RIO CAPIBARIBE


Sem informações para cruzar com os bairros

## 1.3 Logradouros
Este dataset contém a relação de todos os logradouros da cidade de Recife, bem como a definição dos campos dos serviços oferecidos pela prefeitura por face de logradouro.
 - Dataset = http://dados.recife.pe.gov.br/dataset/area-urbana/resource/01187dc5-61d0-438b-b1bc-b1ec508a3e0f
 - Metadados = http://dados.recife.pe.gov.br/dataset/area-urbana/resource/a1ac6fa9-03f6-4843-9f4e-83a658824332

In [7]:
logradouros = pd.read_csv('data/logradouros.csv',sep=';')
logradouros.head(5)

,distrito,quadra,face,codlogradouro,tipologradouro,nomelogradouro,codbairro,nomebairro,valor_v0,codiluminacao,...,desccoleta,codarborizacao,codredeagua,codlimpezaurbana,desclimpezaurbana,codredesgoto,codredetelefonica,codguiasesarjetas,codgaleriapluviais,codredeeletrica
0,1,20,4,388,RUA,Rua General Abreu e Lima,183,ROSARINHO,30.0,3,...,CONVENCIONAL ALTERNA,1,1,2,REGULAR ALTERNADA,1,1,1,1,1
1,1,40,3,388,RUA,Rua General Abreu e Lima,183,ROSARINHO,30.0,3,...,CONVENCIONAL ALTERNA,1,1,2,REGULAR ALTERNADA,1,1,1,1,1
2,1,45,4,388,RUA,Rua General Abreu e Lima,183,ROSARINHO,30.0,3,...,CONVENCIONAL ALTERNA,1,1,2,REGULAR ALTERNADA,1,1,1,1,1
3,1,50,4,388,RUA,Rua General Abreu e Lima,183,ROSARINHO,30.0,3,...,CONVENCIONAL ALTERNA,1,1,2,REGULAR ALTERNADA,1,1,1,1,1
4,1,70,2,388,RUA,Rua General Abreu e Lima,191,TAMARINEIRA,30.0,3,...,CONVENCIONAL ALTERNA,1,1,2,REGULAR ALTERNADA,1,1,1,1,1


In [8]:
# converte tipos para categorico
logradouros['tipologradouro'] = logradouros.tipologradouro.astype('category')
logradouros['desciluminacao'] = logradouros.desciluminacao.astype('category')
logradouros['descpavimentacao'] = logradouros.descpavimentacao.astype('category')
logradouros['desccoleta'] = logradouros.desccoleta.astype('category')
logradouros['codarborizacao'] = logradouros.codarborizacao.astype('category')
logradouros['codredeagua'] = logradouros.codredeagua.astype('category')
logradouros['desclimpezaurbana'] = logradouros.desclimpezaurbana.astype('category')
logradouros['codredesgoto'] = logradouros.codredesgoto.astype('category')
logradouros['codredetelefonica'] = logradouros.codredetelefonica.astype('category')
logradouros['codguiasesarjetas'] = logradouros.codguiasesarjetas.astype('category')
logradouros['codgaleriapluviais'] = logradouros.codgaleriapluviais.astype('category')
logradouros['codredeeletrica'] = logradouros.codredeeletrica.astype('category')

# ajuste de alguns valores
logradouros['codarborizacao'] = logradouros['codarborizacao'].map({2: 0, 1: 1})
logradouros['codredeagua'] = logradouros['codredeagua'].map({2: 0, 1: 1})
logradouros['codredesgoto'] = logradouros['codredesgoto'].map({2: 0, 1: 1})
logradouros['codredetelefonica'] = logradouros['codredetelefonica'].map({2: 0, 1: 1})
logradouros['codguiasesarjetas'] = logradouros['codguiasesarjetas'].map({2: 0, 1: 1})
logradouros['codgaleriapluviais'] = logradouros['codgaleriapluviais'].map({2: 0, 1: 1})
logradouros['codredeeletrica'] = logradouros['codredeeletrica'].map({2: 0, 1: 1})

# drop colunas desnecessarias ou reduntantes
logradouros.drop(['face','valor_v0','codiluminacao','codpavimentacao', 'codcoleta','codlimpezaurbana'], axis=1, inplace=True)

# renomeia algumas colunas por conveniencia
logradouros = logradouros.rename(index=str,columns={"desciluminacao": "iluminacao","descpavimentacao":"pavimentacao",\
                                                    "desccoleta":"coleta", "codarborizacao":"arborizacao",\
                                                    "codredeagua":"redeagua", "desclimpezaurbana":"limpezaurbana",\
                                                    "codredetelefonica":"redetelefonica","codguiasesarjetas":"guiasesarjetas",\
                                                    "codgaleriapluviais":"galeriapluviais","codredeeletrica":"redeeletrica"})

In [9]:
logradouros.groupby('nomebairro')['codlogradouro'].count().describe()

count      45.000000
mean      462.644444
std       591.400885
min         1.000000
25%       127.000000
50%       324.000000
75%       507.000000
max      2640.000000
Name: codlogradouro, dtype: float64

In [19]:
casos_dengue = pd.read_csv('data/casos-dengue2015.csv',sep=';',low_memory=False)

In [20]:
casos_dengue_logradouro = pd.read_csv('logadouro.csv',low_memory=False)
casos_dengue_logradouro = casos_dengue_logradouro.rename(index=str, columns={"nome_logradouro_residencia": "nome"})

Intersecção dos datasets

In [21]:
import unidecode

logradouros.nomelogradouro = logradouros.nomelogradouro.apply(lambda x: x.upper())
logradouros.nomelogradouro = logradouros.nomelogradouro.apply(lambda x: unidecode.unidecode(x))
logradouros.nomelogradouro = logradouros.nomelogradouro.apply(lambda x: x.replace("AVENIDA", "AV"))
logradouros.nomelogradouro = logradouros.nomelogradouro.apply(lambda x: x.replace("AV.", "AV"))
logradouros.nomelogradouro = logradouros.nomelogradouro.apply(lambda x: x.replace("RUA", "R"))

casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("AVENIDA", "AV"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("AV.", "AV"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("RUA", "R"))

#print(casos_dengue_logadouro.head(5))
#logradouros[logradouros.nomelogradouro == "AVENIDA CENTRAL"]

print(casos_dengue_logradouro.head(5))
print(logradouros.nomelogradouro.head(5))

   Unnamed: 0              nome  quantidade
0           0  R ODETE MONTEIRO          35
1           1        AV CENTRAL          29
2           2   R SAO SEBASTIAO          27
3           3    R MASSARANDUBA          26
4           4        R TAPIREMA          25
0    R GENERAL ABREU E LIMA
1    R GENERAL ABREU E LIMA
2    R GENERAL ABREU E LIMA
3    R GENERAL ABREU E LIMA
4    R GENERAL ABREU E LIMA
Name: nomelogradouro, dtype: object


In [34]:
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("1?", "1A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("2?", "2A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("3?", "3A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("4?", "4A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("5?", "5A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("6?", "6A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("7?", "7A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("8?", "8A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("9?", "9A"))

casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("1¬", "1A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("2¬", "2A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("3¬", "3A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("4¬", "4A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("5¬", "5A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("6¬", "6A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("7¬", "7A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("8¬", "8A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("9¬", "9A"))

casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("JOS?", "JOSE"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("F?", "FE"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("CAXANG?", "CAXANGA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("CAXANG?", "CAXANGA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("D?O", "DAO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("Ã?", "CA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("?O", "AO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("S?", "SA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("L?N", "LAN"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("D'?", "D'A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("MAGAL?ES", "MAGALHAES"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("MAGALH?ES", "MAGALHAES"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("?ES", "AES"))

casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("R?GO", "REGO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("PARA?SO", "PARAISO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("ANDR?", "ANDRE"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("JACA?NA", "JACAUNA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("IRAJ?", "IRAJA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("QUIT?RIA", "QUITERIA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("L?RIOS", "LIRIOS"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("ABATI?", "ABATIA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("PAJE?", "PAJEU"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("ITABORA?", "ITABORAI"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("FEL?CIO", "FELICIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("MAUR?CIO", "MAURICIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("JIQUI?", "JIQUIA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("FABR?CIO", "FABRICIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("TOM?", "TOME"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("C?LIA", "CELIA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("OT?VIO", "OTAVIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("GEN?SIO", "GENESIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("TI?MA", "TIUMA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("B?RBARA", "BARBARA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("PATROC?NIO","PATROCINIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("LU?S","LUIS"))

casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("?GUA", "AGUA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("APTROC?NIO", "APTROCINIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("JERUSAL?M","JERUSALEM"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("?MPAR", "IMPAR"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("SA?DE", "SAUDE"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("L?RIO", "LIRIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("JOSU?", "JOSUE"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("MACA?", "MACAE"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("TR?S", "TRES"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("HEMET?RIO", "HEMETERIO"))

casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: unidecode.unidecode(x))

In [35]:
casos_dengue_logradouro = casos_dengue_logradouro.drop(casos_dengue_logradouro[casos_dengue_logradouro.nome.str.contains('?', regex=False)].index)
#logradouros[logradouros.nomelogradouro.str.contains('W. R.', regex=False)]
casos_dengue_logradouro

,Unnamed: 0,nome,quantidade
0,0,R ODETE MONTEIRO,35
1,1,AV CENTRAL,29
2,2,R SAO SEBASTIAO,27
3,3,R MASSARANDUBA,26
4,4,R TAPIREMA,25
5,5,R ZEFERINO AGRA,24
6,6,R TORRES HOMEM,24
7,7,R BAMBU,22
8,8,R ENGENHO NORUEGA,21
9,9,R VINTE E UM DE ABRIL,21


In [79]:
logradouros["CASOS_DENGUE"] = np.nan

In [81]:
#qtd = logradouros[]
#candidatos_2014 = candidatos_2014.assign(QTD_CAPITAL = qtd.capital)

#logradouros["CASOS_DENGUE"] = logradouros["CASOS_DENGUE"].apply(lambda x: x + 1)

#if(have_info[i]):
        #logradouros[i].CASOS_DENGUE = casos_dengue_logradouro[casos_dengue_logradouro.nome == logradouros[i].nomelogradouro].quantidade.values[0]

have_info = logradouros.nomelogradouro.isin(casos_dengue_logradouro.nome.tolist()).tolist()
for i in range(0,len(have_info)-1):
    if have_info[i]:
        logradouros.CASOS_DENGUE.iloc[i] = casos_dengue_logradouro[casos_dengue_logradouro.nome == logradouros.iloc[i].nomelogradouro].quantidade.values[0]     

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [85]:
l2 = logradouros.dropna()
l2.CASOS_DENGUE.describe()

count    6682.000000
mean        4.046992
std         5.434665
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max        29.000000
Name: CASOS_DENGUE, dtype: float64